In [1]:
def crop_right(img):
    # draw filled rectangle in white on black background as mask
    mask = np.zeros_like(img)
    mask = cv2.rectangle(mask,(240,300),(1186,700),(255,255,255),-1)
    # apply mask to image
    result = cv2.bitwise_and(img, mask)
    return result

def crop_left(img):
    # draw filled rectangle in white on black background as mask
    mask = np.zeros_like(img)
    mask = cv2.rectangle(mask,(360,230),(1273,684),(255,255,255),-1)
    # apply mask to image
    result = cv2.bitwise_and(img, mask)
    return result


In [2]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

left_first = cv2.imread('../../rectified_images/left/0.png')
after = cv2.imread('../../rectified_images/left/500.png')


left_first_cropped = crop_left(left_first)
after_cropped = crop_left(after)

# Convert images to grayscale
before_gray = cv2.cvtColor(left_first_cropped, cv2.COLOR_BGR2GRAY)
after_gray = cv2.cvtColor(after_cropped, cv2.COLOR_BGR2GRAY)

before_gray = cv2.blur(before_gray, (10,10))
after_gray = cv2.blur(after_gray, (10,10))
# Compute SSIM between two images
(score, diff) = structural_similarity(before_gray, after_gray, full=True)
print("Image similarity", score)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1]
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

mask = np.zeros(left_first_cropped.shape, dtype='uint8')
filled_after = after_cropped.copy()

for c in contours:
    area = cv2.contourArea(c)
    if area > 2000:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(left_first_cropped, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.rectangle(after_cropped, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.drawContours(mask, [c], 0, (0,255,0), -1)
        cv2.drawContours(filled_after, [c], 0, (0,255,0), -1)

cv2.imshow('before', left_first_cropped)
cv2.imshow('after', after_cropped)
cv2.imshow('diff',diff)
cv2.imshow('mask',mask)
cv2.imshow('filled after',filled_after)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image similarity 0.9663089863603066
